In [1]:
import pandas as pd 
import numpy as np 
import infrastructure
import filepath 
import pygsheets
import datetime as dt
import openpyxl 
import os
from pathlib import Path

# use creds to create a client to interact with the Google Drive API
gc = pygsheets.authorize(service_account_file=filepath.service_account_location)
mamba = gc.open_by_url('https://docs.google.com/spreadsheets/d/12vqSDueybprNphtsw7gXR5vmgcPG6_5ZNcnWzNpiasY/edit#gid=1238872091') 
pipeline_test = gc.open_by_url("https://docs.google.com/spreadsheets/d/16vrHMWs0ambcBJ1sC0SqpYVG8SOSWlbg1N22-bF49v8/edit#gid=1963352944")
pipeline_test_wks = pipeline_test.worksheet('title','Testing Scheduler')
pl_df = pipeline_test_wks.get_as_df()
# open worksheet - mamba
schedule_wks  =  mamba.worksheet('title','New Mamba')
schedule_df = schedule_wks.get_as_df()
#la nina
lanina = infrastructure.get_lanina()
lanina1 =  lanina[~(lanina['Content Approval Status'].str.contains('Paused|Not Approved|Failed Testing', na = False)) ]

In [2]:
# find the row index in the schedule sheet
def find_schedule_row(schedule_df, shortcode):
    matching_rows = schedule_df[schedule_df['Dataset'] == shortcode].index
    return matching_rows[0] if not matching_rows.empty else None

In [3]:
# find the column index for the date
def find_date_column(schedule_wks, test_date):
    for idx, col in enumerate(schedule_wks.get_row(2, include_tailing_empty=False)[3:], start=4):
        if pd.to_datetime(col).date() == test_date.date():
            return idx
    return None

In [4]:
def column_to_letters(column):
    """Convert a column number to a column letter."""
    string = ""
    while column > 0:
        column, remainder = divmod(column - 1, 26)
        string = chr(65 + remainder) + string
    return string

In [5]:
# Iterate over pl_df and update the schedule sheet
for index, row in pl_df.iterrows():
    shortcode = row['Shortcode_DP.SV']
    test_date = pd.to_datetime(row['Testing Date'])
    drop_number = row['Drop Number']
    time = row['Time']
    segment = row['Segment']
    send_strategy = row['Send Strategy']
    
    #add creative id to pl_df
    sc = row['Offer'].split()[4]
    hitpath = row['Offer'].split()[0]
    df_content= lanina1[(lanina1['OfferIDs'] == int(hitpath)) & (lanina1['Type'] == sc) & (lanina1['Channel'] =='SC')][['Reporting Content ID','Content Approval Status', 'Content']]  
    ccid  = df_content['Reporting Content ID'].values[0]
    col_idx = 10
    col_letter = column_to_letters(col_idx)
    cell_address = f"{col_letter}{index+2}"
    pipeline_test_wks.update_value(cell_address, ccid)
    
    offer = row['Offer']
    limit = row['Limit']
    offset = row['Offset']
    creative = row['Creative']
    job_name = row['Job Name']
    p_limit = "Total - "+(str(limit+1))
    p_offset = str(limit+1)
    
    schedule_row_idx = find_schedule_row(schedule_df, shortcode)
    date_col_idx = find_date_column(schedule_wks, test_date)

    if schedule_row_idx is not None and date_col_idx is not None:
        values_to_update = [time, segment, send_strategy, offer, limit, offset, creative, job_name]

        # update cells with new data
        for i, value in enumerate(values_to_update):
            col_letter = column_to_letters(date_col_idx)
            cell_address = f"{col_letter}{schedule_row_idx + 12 + i}"
            schedule_wks.update_value(cell_address, value)
        
        cell_address = f"{col_letter}{schedule_row_idx + 7}"
        schedule_wks.update_value(cell_address, p_limit)
        cell_address = f"{col_letter}{schedule_row_idx + 8}"
        schedule_wks.update_value(cell_address, p_offset)